## Load Resume Dataset from Kaggle
https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset/data

In [1]:
# load data
import pandas as pd
df = pd.read_csv("Resume.csv")
print(df.head())

         ID                                         Resume_str  \
0  16852973           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1  22323967           HR SPECIALIST, US HR OPERATIONS      ...   
2  33176873           HR DIRECTOR       Summary      Over 2...   
3  27018550           HR SPECIALIST       Summary    Dedica...   
4  17812897           HR MANAGER         Skill Highlights  ...   

                                         Resume_html Category  
0  <div class="fontsize fontface vmargins hmargin...       HR  
1  <div class="fontsize fontface vmargins hmargin...       HR  
2  <div class="fontsize fontface vmargins hmargin...       HR  
3  <div class="fontsize fontface vmargins hmargin...       HR  
4  <div class="fontsize fontface vmargins hmargin...       HR  


## Preprocessing 

In [2]:
df.shape

(2484, 4)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [4]:
df['Category'].unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [5]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ADVOCATE                  118
CHEF                      118
ENGINEERING               118
ACCOUNTANT                118
FINANCE                   118
FITNESS                   117
AVIATION                  117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [6]:
# drop column 'Resume_html'
df = df.drop(columns=["Resume_html"])

In [7]:
# show full column width
pd.set_option('display.max_colwidth', None)
# show first row
df["Resume_str"].iloc[0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

In [8]:
import re

# Create a boolean column that marks resumes containing HTML tags
df["has_html"] = df["Resume_str"].apply(lambda x: bool(re.search(r"<.*?>", str(x))))

# Count how many resumes have HTML tags
print("Resumes with HTML tags:", df["has_html"].sum())
print("Resumes without HTML tags:", len(df) - df["has_html"].sum())



Resumes with HTML tags: 10
Resumes without HTML tags: 2474


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          2484 non-null   int64 
 1   Resume_str  2484 non-null   object
 2   Category    2484 non-null   object
 3   has_html    2484 non-null   bool  
dtypes: bool(1), int64(1), object(2)
memory usage: 60.8+ KB


In [10]:
# Drop the 'has_html' column
df = df.drop(columns=['has_html'])

# readability
import re

def clean_text(text):
    text = str(text)
    text = re.sub(r'<.*?>', ' ', text)               # remove HTML tags
    text = re.sub(r'\r', '', text)                   # remove returns
    text = re.sub(r'\n{3,}', '\n\n', text)           # limit multiple newlines to 2
    text = re.sub(r' {2,}', ' ', text)               # collapse extra spaces
    text = re.sub(r'(?<=\w)\n(?=\w)', ' ', text)     # avoid linebreaks mid-sentence
    
    # Add spacing before common section headers
    headers = [
        "summary", "highlights", "experience", "education",
        "projects", "skills", "certifications", "awards", "languages"
    ]
    for h in headers:
        pattern = rf"(?i)(?<=\b){h}\b"
        text = re.sub(pattern, f"\n\n{h.upper()}", text)
    
    text = text.strip()
    return text

# Apply the cleaning function to resumes
df["Resume_clean"] = df["Resume_str"].apply(clean_text)

# Quick check
df[["ID", "Category", "Resume_clean"]].head(3)


ID Category  \
0  16852973       HR   
1  22323967       HR   
2  33176873       HR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

### Entity Extraction (Use  in HR: Skill, Experience,Education etc.)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            2484 non-null   int64 
 1   Resume_str    2484 non-null   object
 2   Category      2484 non-null   object
 3   Resume_clean  2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [12]:
# Filter for HR category only
df_hr = df[df["Category"].str.upper() == "HR"].reset_index(drop=True)

# Check how many HR resumes 
print("Number of HR resumes:", df_hr.shape[0])

# Quick preview
df_hr[["ID", "Category", "Resume_clean"]].head(3)



Number of HR resumes: 110


ID Category  \
0  16852973       HR   
1  22323967       HR   
2  33176873       HR   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [13]:
df_hr.to_csv("hr_resumes.csv", index=False)

In [14]:
# Load Ontology
import json

with open("Ontology.json", "r", encoding="utf-8") as f:
    ontology = json.load(f)

# Check top-level structure
print("Ontology ID:", ontology.get("ontology_id"))
print("Version:", ontology.get("version"))
print("Entities:", list(ontology["entities"].keys()))


Ontology ID: cv_ontology_basic
Version: 1.0.0
Entities: ['education', 'work_experience', 'skills', 'language', 'certifications', 'projects', 'awards', 'voluntary experience', 'publications', 'hobbies']


In [15]:
# view
print(json.dumps(ontology, indent=2))

{
  "ontology_id": "cv_ontology_basic",
  "version": "1.0.0",
  "description": "Ontology for CV extraction (knowledge recall)",
  "entities": {
    "education": {
      "description": "Formal degree or training",
      "attributes": {
        "degree_level": {
          "type": "enum",
          "values": [
            "bachelor",
            "master",
            "phd",
            "diploma",
            "certificate",
            "other"
          ]
        },
        "degree_title": {
          "type": "string"
        },
        "field": {
          "type": "string"
        },
        "year": {
          "type": "integer",
          "optional": true
        }
      },
      "evidence_required": true
    },
    "work_experience": {
      "description": "work experience",
      "attributes": {
        "job_title": {
          "type": "string"
        },
        "company": {
          "type": "string",
          "optional": true
        },
        "start_date": {
          "type": "st

In [16]:
import os

# Create folder for text outputs
os.makedirs("resume_texts", exist_ok=True)

# Randomly sample 20 HR resumes
sample_df = df_hr.sample(n=20, random_state=42)

# Save each resume as a readable text file
for _, row in sample_df.iterrows():
    resume_id = row["ID"]
    category = row["Category"]
    text = row["Resume_clean"]
    
    # Add metadata header
    formatted_text = (
        f"ID: {resume_id}\n"
        f"Category: {category}\n"
        f"{'='*60}\n\n"
        f"{text}"
    )
    
    file_path = f"resume_texts/resume_{resume_id}.txt"
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(formatted_text)


In [17]:

# Folder for annotations
os.makedirs("annotations_manual", exist_ok=True)

# Function to build annotation template
def make_annotation_json(resume_id, category, text, ontology):
    def build_entity_template(entity_def):
        attrs = {}
        for attr, details in entity_def["attributes"].items():
            attrs[attr] = ""  # initialize empty for manual or auto fill
        return attrs

    return {
        "id": str(resume_id),
        "category": category,
        "resume_text": text.strip(),
        **{entity: build_entity_template(defn) for entity, defn in ontology["entities"].items()},
        "role_fit": "",
        "reasoning": ""
    }


# Create and save a JSON file per sampled resume
for _, row in sample_df.iterrows():
    resume_id = row["ID"]
    category = row["Category"]
    text = row["Resume_clean"]

    # Build annotation template
    annotation = make_annotation_json(resume_id, category, text, ontology)
    
    # Save as JSON
    json_path = f"annotations_manual/annotated_{resume_id}.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(annotation, f, indent=2, ensure_ascii=False)

In [51]:
# add two lines to ontology schema

import json

ontology_new = {
  "ontology_id": "cv_ontology_new",
  "version": "1.0.0",
  "description": "Simplified schema for HR resume extraction and reasoning evaluation",
  "schema": {
    "id": "",
    "category": "HR",
    "resume_text": "",
    "education": {
      "degree_level": "",
      "degree_title": "",
      "field": "",
      "year": "",
      "institution": ""
    },
    "work_experience": {
      "job_title": "",
      "company": "",
      "start_date": "",
      "end_date": "",
      "responsibilities": ""
    },
    "skills": {
      "name": "",
      "years_experience": ""
    },
    "language": {
      "name": "",
      "level": ""
    },
    "certifications": {
      "name": "",
      "issuing_organization": "",
      "date_obtained": "",
      "valid_until": ""
    },
    "projects": {
      "name": "",
      "description": "",
      "technologies_used": "",
      "role": "",
      "duration": ""
    },
    "awards": {
      "name": "",
      "issuing_organization": "",
      "date_received": "",
      "description": ""
    },
    "voluntary_experience": {
      "role": "",
      "organization": "",
      "start_date": "",
      "end_date": "",
      "description": ""
    },
    "publications": {
      "title": "",
      "publication_venue": "",
      "date_published": "",
      "url": ""
    },
    "hobbies": {
      "name": "",
      "description": ""
    },
    "role_fit": "",
    "reasoning": ""
  }
}

# Save to file
with open("ontology_new.json", "w", encoding="utf-8") as f:
    json.dump(ontology_new, f, indent=2, ensure_ascii=False)




In [ ]:
# annotate manually > gold standards 
# Prompt for llm 1 (test llm)
""" You are an HR specialist and information extraction expert.

Extract key information from the following resume using the schema below.
Use only information explicitly stated (no assumptions). 
If information is missing, leave it as an empty string. 
Return valid JSON only — no explanations or extra text.

Schema:
{
  "id": "",
  "category": "HR",
  "resume_text": "",
  "education": {
    "degree_level": "",
    "degree_title": "",
    "field": "",
    "year": "",
    "institution": ""
  },
  "work_experience": {
    "job_title": "",
    "company": "",
    "start_date": "",
    "end_date": "",
    "responsibilities": ""
  },
  "skills": {
    "name": "",
    "years_experience": ""
  },
  "language": {
    "name": "",
    "level": ""
  },
  "certifications": {
    "name": "",
    "issuing_organization": "",
    "date_obtained": "",
    "valid_until": ""
  },
  "projects": {
    "name": "",
    "description": "",
    "technologies_used": "",
    "role": "",
    "duration": ""
  },
  "awards": {
    "name": "",
    "issuing_organization": "",
    "date_received": "",
    "description": ""
  },
  "voluntary_experience": {
    "role": "",
    "organization": "",
    "start_date": "",
    "end_date": "",
    "description": ""
  },
  "publications": {
    "title": "",
    "publication_venue": "",
    "date_published": "",
    "url": ""
  },
  "hobbies": {
    "name": "",
    "description": ""
  },
  "role_fit": "",
  "reasoning": ""
}

role_fit must be one of: "strong", "moderate", or "weak".
reasoning: 2–3 sentences explaining your decision based only on explicit resume evidence.

Resume:
{{resume_text}}
 """



In [57]:
# prompt using ollama _ test for 1 resume
# 
# import subprocess
# 
# 
# # Load resume
# with open("resume_texts/resume_11480899.txt", "r", encoding="utf-8") as f:
#     resume_text = f.read()
# 
# #  Load ontology 
# with open("ontology_new.json", "r", encoding="utf-8") as f:
#     ontology = json.load(f)
# 
# #  Prompt 
# prompt = f"""
# You are an HR specialist and information extraction expert.
# 
# Your task:
# Given a candidate’s resume, extract all relevant information according to the provided ontology
# and evaluate their suitability for HR-related roles.
# 
# Follow these rules strictly:
# - Use only information explicitly written in the resume (no assumptions or inferences).
# - Keep responses factual and concise.
# - If information is missing, leave the field empty.
# - Output must be *valid JSON only*, matching the structure and keys in the ontology below.
# - "role_fit" must be one of: "strong", "moderate", or "weak".
# - "reasoning" must be 2–3 sentences explaining your decision based only on explicit evidence from the resume.
# 
# Ontology schema to follow:
# {json.dumps(ontology['schema'], indent=2)}
# 
# Resume text:
# {resume_text}
# """
# 
# # Run Ollama model 
# ollama_path = r"C:\Users\semra\AppData\Local\Programs\Ollama\ollama.exe"
# 
# result = subprocess.run(
#     [ollama_path, "run", "llama3.2:3b"],
#     input=prompt.encode("utf-8"),
#     capture_output=True
# )
# 
# #  Get model output 
# model_output = result.stdout.decode("utf-8").strip()
# 
# # Save to file 
# with open("model_output_11480899.json", "w", encoding="utf-8") as f:
#     f.write(model_output)
# 
# 


In [ ]:
## prompt using ollama _ test for 1 resume
import requests
import json

# Load ontology and resume text
with open("ontology_new.json", "r", encoding="utf-8") as f:
    ontology = json.load(f)

with open("resume_texts/resume_11480899.txt", "r", encoding="utf-8") as f:
    resume_text = f.read()

# prompt
prompt = f"""
You are an HR specialist and information extraction expert.

Your task:
Given a candidate’s resume, extract key information according to the following ontology and then provide reasoning about suitability for HR roles.

Follow these rules:
- Use only what is explicitly written in the resume (no assumptions).
- Keep responses factual and concise.
- If information is missing, leave the field empty.
- Output valid JSON only.
- role_fit must be one of: "strong", "moderate", or "weak".
- reasoning: 2–3 sentences explaining your decision based only on explicit resume evidence.

Ontology schema to follow:
{json.dumps(ontology, indent=2)}

Resume text:
{resume_text}
"""

# Call Ollama API 
url = "http://127.0.0.1:11501/api/generate"

payload = {
    "model": "llama3.2:3b",  
    "prompt": prompt,
}

response = requests.post(url, json=payload, stream=True)

output = ""
for line in response.iter_lines():
    if line:
        data = json.loads(line.decode("utf-8"))
        if "response" in data:
            output += data["response"]

# Save the model output
with open("model_output_11480899.json", "w", encoding="utf-8") as f:
    f.write(output.strip())




In [34]:
# Evaluation using deepval > extraction accuracy (are fields filled correctly) and reasoning quality (logical and faithfulness to gold standard?) and maybe relevance?

from deepeval import evaluate
from deepeval.metrics import JSONMatchMetric, FaithfulnessMetric
import json

# Load gold (ground truth) and model outputs 
with open("annotated_11480899.json", "r", encoding="utf-8") as f:
    gold = json.load(f)

with open("model_output_11480899.json", "r", encoding="utf-8") as f:
    pred = json.load(f)

# Define metrics 
json_match = JSONMatchMetric(
    expected_json=gold,
    predicted_json=pred,
    fields_to_ignore=["reasoning"],  # reasoning is handled separately
    fuzzy_match=True,                # allows partial text match (e.g., capitalization differences)
)

faithfulness = FaithfulnessMetric(
    reference_text=gold["resume_text"],
    prediction_text=pred.get("reasoning", ""),
)

# Run evaluation 
results = evaluate([json_match, faithfulness])

print("DeepEval Results")
for metric, score in results.items():
    print(f"{metric}: {score:.3f}")


